# 14. 체인 (Chains)
- **체인의 생성 단계는 모든 과정을 하나로 묶어 RAG 파이프라인으로 조립하여 완성하는 단계**
- 문서 기반 RAG의 구조도

<img src='https://static.wikidocs.net/images/page/233774/RAG_%E1%84%80%E1%85%AE%E1%84%8C%E1%85%A9%E1%84%83%E1%85%A9_RunnablePassthrough.png' width=800>

<br>

### 코드
- LCEL문법을 사용하여 7단계의 전 과정을 하나의 체인으로 묶음

```python
# 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
```

<br>

- 완성된 체인에 질의하는 코드

```python
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력
question = "삼성전자가 자체 개발한 AI 의 이름은?"
response = chain.invoke(question)
```

<br>

<hr>

<br>

## 01. 문서 요약

<br>

#### 주요 개요
1. **Stuff**: 전체 문서 한 번에 요약
2. **Map-Reduce**: 분할 요약 후 일괄 병합
3. **Map-Refine**: 분할 요약 후 점진적인 병합
4. **Chain of Density**: N번 반복 실행하며, 누락된 entity를 보완하며 요약 개선
5. **Clustering-Map-Refine**: 문서의 Chunk 를 N 개의 클러스터로 나누고, 각 클러스터에서 중심점에 가까운 문서에 대한 요약을 Refine 요약

<br>

### 대표적인 요약 방식
- 요약기를 구축할 떄 중심적인 질문은 **문서를 LLM 컨텍스트 창에 어떻게 전달할 것인가**

1. **`Stuff`** : 단순히 모든 문서를 단일 프롬프트로 '넣는' 방식 (가장 단순)
2. **`Map-Reduce`**: 각 문서를 `map` 단계에서 개별적으로 요약한 다음, `reduce` 단계에서 요약본들을 최종 요약본으로 합침
3. **`Refine`** : 입력 문서를 순회하며 반복적으로 답변을 업데이트하며 응답을 구성. 각 문서에 대해 모든 비문서 입력, 현재 문서, 그리고 최신 중간 답변을 `chian`에 전달하여 새로운 답변을 얻음

<br>


In [1]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

<br>

#### `Stuff`
- `stuff documents chain`("stuff"는 "채우다" 또는 "채우기 위해"의 의미)는 문서 체인 중 가장 간단한 방식
- **문서 목록을 가져와서 모두 프롬프트에 삽입한 다음, 그 프롬프트를 LLM에 전달**
- **이 체인은 문서가 작고 대부분의 호출에 몇 개만 전달되는 애플리케이션에 적합**

In [2]:
from langchain_community.document_loaders import TextLoader

In [4]:
loader = TextLoader("data/news.txt")
docs = loader.load()
print(f"총 글자수: {len(docs[0].page_content)}")
print("\n========= 앞부분 미리보기 =========\n")
print(docs[0].page_content[:500])

총 글자수: 1708

========= 앞부분 미리보기 =========

제목: 
AI2, 상업 활용까지 자유로운 '진짜' 오픈 소스 LLM '올모' 출시

내용:
앨런AI연구소(AI2)가 완전한 오픈 소스 대형언어모델(LLM) '올모(OLMo)’를 출시했다. 데이터 수집, 학습, 배포의 전 과정을 투명하게 공개한 데다 상업적 사용까지 허용한 진정한 의미의 오픈 소스 LLM이라는 평가다.
벤처비트는 1일(현지시간) 비영리 민간 AI 연구기관인 AI2가 ‘최초의 진정한 오픈 소스 LLM 및 프레임워크’라고 소개한 ‘올모’를 출시했다고 보도했다. 
이에 따르면 올모는 모델 코드와 모델 가중치뿐만 아니라 훈련 코드, 훈련 데이터, 관련 툴킷 및 평가 툴킷도 제공한다. 이를 통해 모델이 어떻게 구축되었는지 심층적으로 분석, LLM의 작동 방식과 응답을 생성하는 원리를 더 잘 이해할 수 있다. 
올모 프레임워크는 70억 매개변수의 ‘올모 7B’ 등 4가지 변형 모델과 10억 매개변수의 ‘올모 1B’ 모델을 제공한다. 모델들은 훈련 데이터를 생성하는 코드를 포함해 


- 한국어로 요약을 작성하라는 Prompt

In [6]:
from langchain_classic import hub

In [7]:
prompt = hub.pull("teddynote/summary-stuff-documents-korean")
prompt.pretty_print()

Please summarize the sentence according to the following REQUEST.
REQUEST:
1. Summarize the main points in bullet points in KOREAN.
2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.
3. Use various emojis to make the summary more interesting.
4. Translate the summary into KOREAN if it is written in ENGLISH.
5. DO NOT translate any technical terms.
6. DO NOT include any unnecessary information.

CONTEXT:
{context}

SUMMARY:"



In [ ]:
from langchain_openai import ChatOpenAI
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
# from langchain_teddynote.callbacks import StreamingCallback

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    streaming=True,
    temperature=0,
    callbacks=[StreamingStdOutCallbackHandler()],
)

stuff_chain = create_stuff_documents_chain(llm, prompt)
answer = stuff_chain.invoke({"context": docs})

- 🚀 앨런AI연구소(AI2)가 완전한 오픈 소스 LLM '올모(OLMo)'를 출시했다.  
- 📊 데이터 수집, 학습, 배포 과정을 투명하게 공개하고 상업적 사용을 허용한다.  
- 🛠️ 모델 코드, 가중치, 훈련 코드, 데이터 및 평가 툴킷을 제공한다.  
- 📈 '올모 7B'와 '올모 1B' 등 4가지 변형 모델을 포함한다.  
- 🔍 AI2의 '돌마(Dolma)' 데이터 세트를 기반으로 3조개의 토큰으로 훈련되었다.  
- 📜 아파치 2.0 라이선스에 따라 상업적 활용에 제한이 없다.  
- 🧪 연구자들이 모델의 작동 방식을 과학적으로 이해할 수 있도록 돕는다.  
- 🌟 올모는 상업용 제품과 동등한 성능을 보여준다.  
- ⚠️ 비영어권 언어에 대한 낮은 품질과 약한 코드 생성 기능이 있다.  
- 🔄 AI2는 올모를 계속해서 향상할 계획이다.  
- 🌐 올모의 모든 리소스는 깃허브 및 허깅페이스에서 무료로 제공된다.  

<br>

### Map-Reduce
- **Map-Reduce 방식의 요약은 긴 문서를 효율적으로 요약하는 기법**
- **이 방법은 문서를 작은 `chunk`로 나누는 `map` 단계와, 각 `chunk`의 요약을 결합하는 `reduce` 단계로 결합**

1. `Map` 단계에서는 각 `chunk`를 병렬로 요약
2. `Reduce` 단계에서는 이 요약들을 하나의 최종 요약으로 통합

- **이 접근법은 대규모 문서를 처리할 때 특히 유용하며, 언어 모델의 토큰 제한을 우회**

<br>

<img src='https://static.wikidocs.net/images/page/234020/summarization_use_case_2.png' width=800>

In [14]:
from langchain_community.document_loaders import PyPDFLoader

In [15]:
loader = PyPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()
docs = docs[3:8]  # 여기서 문서의 일부만 요약
print(f"총 페이지수: {len(docs)}")

총 페이지수: 5


<br>

#### Map
- **`Map` 단계에서는 각 Chunk 에 대한 요약을 생성 (혹은 핵심 내용 추출. 마지막 `reduce` 단계에서 요약을 하나로 합치는 과정이기 때문)**

In [16]:
from langchain_classic import hub
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [17]:
llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-4o-mini",
)

# map prompt 다운로드
map_prompt = hub.pull("teddynote/map-prompt")

# 프롬프트 출력
map_prompt.pretty_print()

================================ System Message ================================

You are a professional main thesis extractor.

================================ Human Message =================================

Your task is to extract main thesis from given documents. Answer should be in same language as given document. 

#Format: 
- thesis 1
- thesis 2
- thesis 3
- ...

Here is a given document: 
{doc}

Write 1~5 sentences.
#Answer:


<br>

- **`map_chain` 생성**

In [19]:
map_chain = map_prompt | llm | StrOutputParser()

<br>

- **`batch()` 를 호출하여 각 문서에 대한 요약본을 생성**

In [20]:
doc_summaries = map_chain.batch(docs)

- 요약된 문서의 수

In [21]:
len(doc_summaries)

5

<br>

- 일부 문서의 요약 출력

In [22]:
print(doc_summaries[0])

- 미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행정명령을 발표하였다.
- 이 행정명령은 AI의 안전과 보안 기준 마련, 개인정보보호, 형평성과 시민권 향상, 소비자 보호, 노동자 지원, 혁신과 경쟁 촉진, 국제협력을 포함한다.
- AI 시스템의 안전성과 신뢰성을 확인하기 위한 표준 및 모범사례를 확립하고, AI의 무책임한 사용으로 인한 차별과 편견을 방지하기 위한 조치를 확대할 예정이다.
- 또한, 국가 차원에서 AI 연구 인프라를 확충하고 중소기업과 개발자에게 기술과 인프라를 지원하여 AI 연구를 촉진할 계획이다.


<br>

#### Reduce
- `Reduce` 단계에서는 `map` 단계에서 진행한 핵심 내용들을 하나의 최종 요약으로 통합

In [23]:
reduce_prompt = hub.pull("teddynote/reduce-prompt")

In [24]:
reduce_prompt.pretty_print()

================================ System Message ================================

You are a professional summarizer. You are given a list of summaries of documents and you are asked to create a single summary of the documents.

================================ Human Message =================================

#Instructions: 
1. Extract main points from a list of summaries of documents
2. Make final summaries in bullet points format.
3. Answer should be written in {language}.

#Format: 
- summary 1
- summary 2
- summary 3
- ...

Here is a list of summaries of documents: 
{doc_summaries}

#SUMMARY:


<br>

- **`Reduce Chain` 생성**

In [25]:
reduce_chain = reduce_prompt | llm | StrOutputParser()

In [30]:
for chunk in reduce_chain.stream({"doc_summaries": doc_summaries, "language": "Korean"}):
    print(chunk, end="", flush=True)

- 미국 바이든 대통령이 AI 개발과 사용의 안전성을 보장하기 위한 행정명령을 발표하였으며, 이는 AI의 안전 기준, 개인정보 보호, 형평성 향상 등을 포함한다.
- G7은 '히로시마 AI 프로세스'를 통해 AI 국제 행동강령에 합의하고, AI 시스템의 투명성과 책임성을 강화할 것을 제안하였다.
- 영국 AI 안전성 정상회의에서 블레츨리 선언이 발표되었으며, AI 시스템의 안전성을 보장하기 위한 협력의 중요성이 강조되었다.
- 미국 캘리포니아 북부지방법원은 생성 AI 기업에 대한 저작권 소송을 기각하였고, 원고에게 고소장 수정을 요청하였다.
- 미국 연방거래위원회(FTC)는 생성 AI의 소비자 및 창작자에 대한 피해 가능성을 우려하며, 시장 지배력 강화에 대한 경고를 발령하였다.

In [31]:
from langchain_core.runnables import chain

In [ ]:
@chain
def map_reduce_chain(docs):
    map_llm = ChatOpenAI(
        temperature=0,
        model_name="gpt-4o-mini",
    )

    # map prompt 다운로드
    map_prompt = hub.pull("teddynote/map-prompt")

    # map chain 생성
    map_chain = map_prompt | map_llm | StrOutputParser()

    # 첫 번째 프롬프트, ChatOpenAI, 문자열 출력 파서를 연결하여 체인을 생성
    doc_summaries = map_chain.batch(docs)

    # reduce prompt 다운로드
    reduce_prompt = hub.pull("teddynote/reduce-prompt")
    reduce_llm = ChatOpenAI(
        model_name="gpt-4o",
        temperature=0,
        callbacks=[StreamingStdOutCallbackHandler()],
        streaming=True,
    )

    reduce_chain = reduce_prompt | reduce_llm | StrOutputParser()

    return reduce_chain.invoke({"doc_summaries": doc_summaries, "language": "Korean"})

In [33]:
answer = map_reduce_chain.invoke(docs)

- 미국 바이든 대통령은 AI의 안전하고 신뢰할 수 있는 개발과 사용을 보장하기 위한 행정명령을 발표하였으며, AI의 안전성, 개인정보보호, 형평성, 소비자 보호, 노동자 지원, 혁신 촉진 등을 포함한다.
- G7은 '히로시마 AI 프로세스'를 통해 AI 국제 행동강령에 합의하였으며, AI의 위험 평가, 투명성, 책임성, 정보공유, 이해관계자 협력을 요구하고, AI 생성 콘텐츠의 신뢰성을 보장하기 위한 메커니즘 개발을 강조하였다.
- 영국 AI 안전성 정상회의에서 블레츨리 선언이 발표되었으며, AI 안전성을 보장하기 위해 이해관계자의 협력과 첨단 AI 개발 기업의 책임을 강조하였다. 영국은 AI 안전 연구소 출범과 안전 테스트 계획을 발표하였다.
- 미국 캘리포니아 북부지방법원은 예술가들이 제기한 생성 AI 기업에 대한 저작권 소송을 기각하였으나, 특정 작품에 대한 저작권 침해 소송은 계속 진행 중이다.
- 미국 연방거래위원회(FTC)는 생성 AI로 인한 소비자와 창작자의 피해 가능성 및 빅테크의 시장 지배력 강화 우려를 표명하며, 소비자 보호와 공정한 경쟁 시장 유지를 위해 모든 법적 권한을 활용할 것을 강조하였다.

<br>

### Map-Refine

1. **`Map` 단계** : 문서를 여러 개의 작은 `chunk`로 나누고, 각 `chunk`에 대해 개별적으로 요약을 생성
2. **`Refine` 단계** : 생성된 요약들을 순차적으로 처리하며 최종 요약을 점진적으로 개선. 각 단계에서 이전 요약과 새로운 `chunk`의 정보를 결합하여 요약을 갱신
3. **반복 과정** : 모든 `chunk`가 처리될 때까지 `Refine`단계를 반복
4. **최종 요약** : 마지막 `chunk`까지 처리한 후 얻은 요약이 최종 결과

<br>

- **Map-Refine 방식의 장점은 문서의 순서를 유지하면서 점진적으로 요약을 개선할 수 있다는 것 $\rightarrow$ 이는 특히 문서의 맥락이 중요한 경우에 유용**
- **그러나 이 방식은 Map-Reduce 에 비해 순차적으로 처리되기 때문에 병렬화가 어려워 대규모 문서 처리 시 시간이 더 오래 걸릴 수 있음**

<img src='https://static.wikidocs.net/images/page/234020/summarization_use_case_3.png' width=800>

<br>

#### Map
- `Map` 단계에서 각 chunk에 대한 요약을 생성


In [34]:
from langchain_classic import hub
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [35]:
# map llm 생성
map_llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-4o-mini",
)

# map chain 생성
map_summary = hub.pull("teddynote/map-summary-prompt")

# 프롬프트 출력
map_summary.pretty_print()

================================ System Message ================================

You are an expert summarizer. Your task is to summarize the following document in {language}.

================================ Human Message =================================

Extract most important main thesis from the documents, then summarize in bullet points.

#Format:
- summary 1
- summary 2
- summary 3
-...

Here is a given document: 
{documents}

Write 1~5 sentences. Think step by step.
#Summary:


- `map_chain`을 생성

In [36]:
map_chain = map_summary | llm | StrOutputParser()

- 첫 문서의 요약

In [37]:
print(map_chain.invoke({"documents": docs[0], "language": "Korean"}))

- 미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 위한 행정명령을 발표하였다.
- 이 행정명령은 AI의 안전과 보안 기준, 개인정보 보호, 형평성과 시민권 향상, 소비자 보호, 노동자 지원, 혁신과 경쟁 촉진, 국제 협력을 포함한다.
- AI 시스템 개발 기업은 안전 테스트 결과와 시스템 정보를 미국 정부와 공유해야 하며, AI의 책임 있는 사용을 위한 법적 지침이 마련된다.
- 소비자 보호와 근로자 지원을 위해 의료 분야에서의 AI 사용 촉진 및 교육 도구 개발이 강조된다.
- 국가 차원에서 AI 연구 인프라를 확충하고 외국인 전문가의 유입을 지원하는 방안도 포함되어 있다.


- 모든 문서를 입력으로 정의

In [38]:
input_doc = [{"documents": doc, "language": "Korean"} for doc in docs]

In [39]:
print(map_chain.batch(input_doc))

['- 미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 위한 행정명령을 발표하였다.\n- 이 행정명령은 AI의 안전과 보안 기준, 개인정보 보호, 형평성과 시민권 향상, 소비자 보호, 노동자 지원, 혁신과 경쟁 촉진, 국제 협력을 포함한다.\n- AI 시스템 개발 기업은 안전 테스트 결과와 시스템 정보를 정부와 공유해야 하며, AI의 책임 있는 사용을 위한 법적 지침이 마련된다.\n- 소비자 보호와 근로자 지원을 위해 의료 분야에서의 AI 사용 촉진 및 교육 도구 개발이 강조된다.\n- 국가 차원에서 AI 연구 인프라를 확충하고 외국인 전문가의 유입을 지원하는 방안도 포함되어 있다.', "- G7은 2023년 10월 30일 '히로시마 AI 프로세스'를 통해 첨단 AI 시스템을 개발하는 기업을 위한 국제 행동강령에 합의하였다.\n- 이 행동강령은 AI 수명주기 전반에 걸친 위험 평가 및 완화, 투명성과 책임성 보장, 정보 공유 및 협력, 보안 통제, 콘텐츠 인증 및 출처 확인 등의 조치를 요구한다.\n- G7은 AI 시스템의 성능과 한계를 공개하고, 적절한 사용 영역을 알리며, 강력한 보안 통제를 구현할 것을 권장하고 있다.\n- 또한, AI 생성 콘텐츠의 신뢰성을 높이기 위해 기술적 인증 메커니즘을 개발하고, 사회적 위험을 완화하기 위한 연구에 우선 투자할 것을 강조하고 있다.\n- 이 행동강령은 빠르게 발전하는 기술에 대응하기 위해 이해관계자 협의를 통해 필요에 따라 개정될 예정이다.", '- 28개국이 참여한 영국 AI 안전성 정상회의에서 AI 안전 보장을 위한 블레츨리 선언이 발표되었다.\n- 선언은 AI 시스템의 안전성을 보장하기 위해 모든 이해관계자의 협력이 중요하다고 강조하며, 첨단 AI 개발 기업의 책임을 지적했다.\n- 영국 총리는 AI 안전 연구소의 출범을 발표하고, 정부 주도의 안전 테스트 계획을 수립할 것이라고 밝혔다.\n- 참가국들은 AI의 위험과 가능성에 대한 과학적 평가를 위한 보고서 작성을 합의하고, 한국과 프랑스와

<br>

#### Refine
- 이전의 `Map` 단계에서 생성한 chunk들을 순차적으로 처리하며 최종 요약을 점진적으로 개선
  

In [40]:
refine_prompt = hub.pull("teddynote/refine-prompt")
refine_prompt.pretty_print()

================================ System Message ================================

You are an expert summarizer.

================================ Human Message =================================

Your job is to produce a final summary

We have provided an existing summary up to a certain point:
{previous_summary}

We have the opportunity to refine the existing summary(only if needed) with some more context below.
------------
{current_summary}
------------
Given the new context, refine the original summary in {language}.
If the context isn't useful, return the original summary.


<br>

- `refine_llm` 생성

In [41]:
refine_llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-4o-mini",
)

# refine chain 생성
refine_chain = refine_prompt | refine_llm | StrOutputParser()

<br>

- `map_reduce_chain` 생성 예시
  

In [42]:
from langchain_core.runnables import chain

In [43]:
@chain
def map_refine_chain(docs):

    # map chain 생성
    map_summary = hub.pull("teddynote/map-summary-prompt")

    map_chain = (
        map_summary
        | ChatOpenAI(
            model_name="gpt-4o-mini",
            temperature=0,
        )
        | StrOutputParser()
    )

    input_doc = [{"documents": doc.page_content, "language": "Korean"} for doc in docs]

    # 첫 번째 프롬프트, ChatOpenAI, 문자열 출력 파서를 연결하여 체인을 생성
    doc_summaries = map_chain.batch(input_doc)

    refine_prompt = hub.pull("teddynote/refine-prompt")

    refine_llm = ChatOpenAI(
        model_name="gpt-4o-mini",
        temperature=0,
        callbacks=[StreamingStdOutCallbackHandler()],
        streaming=True,
    )

    refine_chain = refine_prompt | refine_llm | StrOutputParser()

    previous_summary = doc_summaries[0]

    for current_summary in doc_summaries[1:]:

        previous_summary = refine_chain.invoke(
            {
                "previous_summary": previous_summary,
                "current_summary": current_summary,
                "language": "Korean",
            }
        )
        print("\n\n-----------------\n\n")

    return previous_summary

In [44]:
refined_summary = map_refine_chain.invoke(docs)

- 미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 위한 행정명령을 발표하였다.
- 이 행정명령은 AI의 안전과 보안 기준, 개인정보 보호, 형평성과 시민권 향상, 소비자 보호, 노동자 지원, 혁신과 경쟁 촉진, 국제 협력을 포함한다.
- AI 시스템 개발 기업은 안전 테스트 결과와 시스템 정보를 정부와 공유해야 하며, AI의 책임 있는 사용을 위한 법적 지침이 마련된다.
- 의료 분야에서의 AI 사용 촉진과 학교 내 AI 교육 도구 개발을 통해 소비자와 근로자를 보호하는 원칙이 설정된다.
- 국가 차원에서 AI 연구 인프라를 확충하고 외국인 전문가의 유입을 지원하여 AI 분야의 혁신을 촉진할 계획이다.
- 또한, G7은 '히로시마 AI 프로세스'를 통해 AI 기업을 위한 국제 행동강령에 합의하였으며, 이는 AI 시스템의 위험 식별과 완화를 위한 자발적 채택을 권장하고, 위험 평가와 투명성, 책임성을 강조한다.
- G7은 AI 기술의 빠른 발전에 대응하기 위해 행동강령을 개정할 계획이며, 사회적 위험 완화와 세계적 문제 해결을 위한 AI 시스템 개발을 우선시하고 국제 기술 표준의 개발을 가속화할 예정이다.

-----------------


- 미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 위한 행정명령을 발표하였다.
- 이 행정명령은 AI의 안전과 보안 기준, 개인정보 보호, 형평성과 시민권 향상, 소비자 보호, 노동자 지원, 혁신과 경쟁 촉진, 국제 협력을 포함한다.
- AI 시스템 개발 기업은 안전 테스트 결과와 시스템 정보를 정부와 공유해야 하며, AI의 책임 있는 사용을 위한 법적 지침이 마련된다.
- 의료 분야에서의 AI 사용 촉진과 학교 내 AI 교육 도구 개발을 통해 소비자와 근로자를 보호하는 원칙이 설정된다.
- 국가 차원에서 AI 연구 인프라를 확충하고 외국인 전문가의 유입을 지원하여 AI 분야의 혁신을 촉진할 계획이다.
- G7은 '히로시마 AI 프로세스'를 통해 AI 기업을 위한 국제 행동강령에 합의

<br>

### Chain of Density
- https://arxiv.org/pdf/2309.04269
- **Chain of Density" (CoD) 프롬프트는 GPT-4를 사용한 요약 생성을 개선하기 위해 개발된 기법**
  - 이 방법은 초기에 개체가 적은 요약을 생성한 후, 길이를 늘리지 않으면서 누락된 중요 개체들을 반복적으로 통합하는 과정을 거침. 
  - 연구 결과, CoD로 생성된 요약은 일반 프롬프트보다 더 추상적이고 정보 융합이 뛰어나며, 인간이 작성한 요약과 비슷한 밀도를 가진 것으로 나타남

<br>

- **점진적 개선**: CoD는 초기에 개체가 적은 간단한 요약을 생성한 후, 단계적으로 중요한 개체들을 추가하며 요약을 개선
  - 이 과정에서 요약의 길이는 유지되면서 정보 밀도가 증가하여 읽기 쉬우면서도 정보량이 풍부한 요약이 생성

- **정보 밀도와 가독성의 균형**: CoD 방식은 요약의 정보 밀도를 조절하여 정보성과 가독성 사이의 최적 균형점을 탐색
  - 연구 결과에 따르면, 사람들은 일반적인 GPT-4 요약보다 더 밀도 있지만 사람이 작성한 요약만큼 밀도가 높지 않은 CoD 요약을 선호하는 것으로 나타남

- **추상화와 정보 융합 개선**: CoD로 생성된 요약은 더 추상적이고 정보 융합이 뛰어나며, 원문의 앞부분에 치우치는 경향(lead bias)이 덜함. 
  - 이는 요약의 전반적인 품질과 가독성을 향상시키는 데 기여

<br>

#### Chain of Density Prompt 입력 파라미터
- **`content_category`**: 콘텐츠 종류(예: 기사, 동영상 녹취록, 블로그 게시물, 연구 논문). 기본값: Article
- **`content`**: 요약할 콘텐츠
- **`entity_range`**: 콘텐츠에서 선택하여 요약에 추가할 엔티티의 수의 범위. 기본값은 1-3
- **`max_words`**: 1번 요약시, 요약에 포함할 최대 단어. 기본값은 80
- **`iterations`**: 엔티티 고밀도화 라운드 수. 총 요약은 반복 횟수+1 
  - 80단어의 경우 3회 반복이 이상적
  - 요약이 더 길면 4~5회, 그리고 `entity_range`를 1~4로 변경하는 것도 유용
  - 기본값: 3
  - 이 코드는 Chain of Density 프롬프트를 사용하여 텍스트 요약을 생성하는 체인을 구성

<br>



In [45]:
import textwrap
from langchain_classic import hub
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import SimpleJsonOutputParser

- `{content}`를 제외한 모든 입력에 대한 기본값 지정

In [46]:
cod_chain_inputs = {
    "content": lambda d: d.get("content"),
    "content_category": lambda d: d.get("content_category", "Article"),
    "entity_range": lambda d: d.get("entity_range", "1-3"),
    "max_words": lambda d: int(d.get("max_words", 80)),
    "iterations": lambda d: int(d.get("iterations", 5)),
}

- Chain of Density 프롬프트 다운로드

In [56]:
cod_prompt = hub.pull("teddynote/chain-of-density-prompt")
cod_prompt.pretty_print()

================================ System Message ================================

As an expert copy-writer, you will write increasingly concise, entity-dense summaries of the user provided {content_category}. The initial summary should be under {max_words} words and contain {entity_range} informative Descriptive Entities from the {content_category}.

A Descriptive Entity is:
- Relevant: to the main story.
- Specific: descriptive yet concise (5 words or fewer).
- Faithful: present in the {content_category}.
- Anywhere: located anywhere in the {content_category}.

# Your Summarization Process
- Read through the {content_category} and the all the below sections to get an understanding of the task.
- Pick {entity_range} informative Descriptive Entities from the {content_category} (";" delimited, do not add spaces).
- In your output JSON list of dictionaries, write an initial summary of max {max_words} words containing the Entities.
- You now have `[{"missing_entities": "...", "denser_summa

- Chain of Density 체인 생성


In [48]:
cod_chain = (
    cod_chain_inputs
    | cod_prompt
    | ChatOpenAI(temperature=0, model="gpt-4o-mini")
    | SimpleJsonOutputParser()
)

- 두 번째 체인 생성, 최종 요약만 추출

In [49]:
cod_final_summary_chain = cod_chain | (
    lambda output: output[-1].get(
        "denser_summary", '오류: 마지막 딕셔너리에 "denser_summary" 키가 없습니다'
    )
)

In [51]:
content = docs[1].page_content

In [52]:
# 결과를 저장할 빈 리스트 초기화
results: list[dict[str, str]] = []

In [53]:
# cod_chain을 스트리밍 모드로 실행하고 부분적인 JSON 결과를 처리
for partial_json in cod_chain.stream(
    {"content": content, "content_category": "Article"}
):
    # 각 반복마다 results를 업데이트
    results = partial_json

    # 현재 결과를 같은 줄에 출력 (캐리지 리턴을 사용하여 이전 출력을 덮어씀)
    print(results, end="\r", flush=True)

In [54]:
# 총 요약 수 계산
total_summaries = len(results)
print("\n")

# 각 요약을 순회하며 처리
i = 1
for cod in results:
    # 누락된 엔티티들을 추출하고 포맷팅
    added_entities = ", ".join(
        [
            ent.strip()
            for ent in cod.get(
                "missing_entities", 'ERR: "missing_entiies" key not found'
            ).split(";")
        ]
    )
    # 더 밀도 있는 요약 추출
    summary = cod.get("denser_summary", 'ERR: missing key "denser_summary"')

    # 요약 정보 출력 (번호, 총 개수, 추가된 엔티티)
    print(
        f"### CoD Summary {i}/{total_summaries}, 추가된 엔티티(entity): {added_entities}"
        + "\n"
    )
    # 요약 내용을 80자 너비로 줄바꿈하여 출력
    print(textwrap.fill(summary, width=80) + "\n")
    i += 1

print("\n============== [최종 요약] =================\n")
print(summary)




### CoD Summary 1/5, 추가된 엔티티(entity): G7, AI 국제 행동강령, 히로시마 AI 프로세스

G7은 2023년 10월 30일 히로시마 AI 프로세스를 통해 AI 기업을 위한 AI 국제 행동강령을 합의했다. 이 행동강령은 AI 위험 식별과
완화를 위한 자발적 채택을 권장하며, AI 수명주기 전반의 위험 평가, 투명성, 책임성, 정보 공유 및 보안 통제를 요구한다.

### CoD Summary 2/5, 추가된 엔티티(entity): AI 위험 식별, 투명성, 정보 공유

G7은 2023년 10월 30일 히로시마 AI 프로세스를 통해 AI 기업을 위한 AI 국제 행동강령을 합의했다. 이 행동강령은 AI 위험 식별과
완화를 위한 자발적 채택을 권장하며, AI 수명주기 전반의 위험 평가, 투명성 및 책임성, 정보 공유와 보안 통제를 요구한다.

### CoD Summary 3/5, 추가된 엔티티(entity): AI 수명주기, 보안 통제, AI 거버넌스

G7은 2023년 10월 30일 히로시마 AI 프로세스를 통해 AI 기업을 위한 AI 국제 행동강령을 합의했다. 이 행동강령은 AI 위험 식별과
완화를 위한 자발적 채택을 권장하며, AI 수명주기 전반의 위험 평가, 투명성, 정보 공유, 보안 통제 및 AI 거버넌스를 요구한다.

### CoD Summary 4/5, 추가된 엔티티(entity): 물리보안, 사이버보안, 신뢰할 수 있는 콘텐츠

G7은 2023년 10월 30일 히로시마 AI 프로세스를 통해 AI 기업을 위한 AI 국제 행동강령을 합의했다. 이 행동강령은 AI 위험 식별과
완화를 위한 자발적 채택을 권장하며, AI 수명주기 전반의 위험 평가, 투명성, 정보 공유, 물리보안, 사이버보안 및 신뢰할 수 있는 콘텐츠
인증을 요구한다.

### CoD Summary 5/5, 추가된 엔티티(entity): 기후 위기, 세계 보건, 교육

G7은 2023년 10월 30일 히로시마 AI 프로세스를 통해 AI 기업을 위한 AI 국제 행

In [55]:
print(summary)

G7은 2023년 10월 30일 히로시마 AI 프로세스를 통해 AI 기업을 위한 AI 국제 행동강령을 합의했다. 이 행동강령은 AI 위험 식별과 완화를 위한 자발적 채택을 권장하며, AI 수명주기 전반의 위험 평가, 투명성, 정보 공유, 물리보안, 사이버보안, 신뢰할 수 있는 콘텐츠 인증 및 기후 위기, 세계 보건, 교육 문제 해결을 요구한다.


<br>

### Clustering-Map-Refine
- **Map-Reduce나 Map-Refine 방식은 모두 시간이 오래 걸리고, 비용이 많이듬**
    
    $\rightarrow$ **문서를 몇 개 (N 개)의 클러스터로 나눈 뒤, 가장 중심축에서 가까운 문서를 클러스터의 대표 문서로 인지하고, 이를 Map-Reduce(혹은 Map-Refine)방식으로 요약**

In [59]:
from langchain_community.document_loaders import PyMuPDFLoader

In [62]:
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()
len(docs)

23

- 하나의 문서로 텍스트를 합침 $\rightarrow$ page별로 구분하지 않기 위함

In [63]:
texts = "\n\n".join([doc.page_content for doc in docs])
len(texts)

27977

- `RecursiveCharacterTextSplitter` 를 사용하여 하나의 Text 를 여러 문서로 나눔

In [64]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [66]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_docs = text_splitter.split_text(texts)

In [68]:
# 나누어진 문서의 수를 확인
len(split_docs)

79

<br>

- `Upstage Embeddings` 모델을 사용하여 문서를 임베딩

```python
from langchain.upstage import UpstageEmbeddings

embeddings = UpstageEmbeddings(model="solar-embedding-1-large-passage")
vectors = embeddings.embed_documents(split_docs)
```

<br>

- 총 79개의 문서를 10개 클러스터로 나누고, 이때 KMeans 를 사용하여 클러스터링을 수행

```python
from sklearn.cluster import KMeans

num_clusters = 10
kmeans = KMeans(n_clusters=num_clusters, random_state=123).fit(vectors)
```

```python
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

tsne = TSNE(n_components=2, random_state=42)
reduced_data_tsne = tsne.fit_transform(np.array(vectors))

# seaborn 스타일 설정
sns.set_style("white")

# 축소된 데이터 플롯
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x=reduced_data_tsne[:, 0],
    y=reduced_data_tsne[:, 1],
    hue=kmeans.labels_,
    palette="deep",
    s=100,
)
plt.xlabel("Dimension 1", fontsize=12)
plt.ylabel("Dimension 2", fontsize=12)
plt.title("Clustered Embeddings", fontsize=16)
plt.legend(title="Cluster", title_fontsize=12)

# 배경색 설정
plt.gcf().patch.set_facecolor("white")

plt.tight_layout()
plt.show()
```

<img src='images/clustered_embeddings.png' width=800>

<br>

- 각 cluster 의 중심점에 가장 가까운 임베딩을 찾아서 저장

```python
import numpy as np

# 가장 가까운 점들을 저장할 빈 리스트 생성
closest_indices = []

# 클러스터 수만큼 반복
for i in range(num_clusters):

    # 해당 클러스터 중심으로부터의 거리 목록 구하기
    distances = np.linalg.norm(vectors - kmeans.cluster_centers_[i], axis=1)

    # 가장 가까운 점의 인덱스 찾기 (argmin을 사용하여 최소 거리 찾기)
    closest_index = np.argmin(distances)

    # 해당 인덱스를 가장 가까운 인덱스 리스트에 추가
    closest_indices.append(closest_index)
```

<br>

- 문서의 요약을 순서대로 진행하기 위하여 오름차순 정렬

```python
selected_indices = sorted(closest_indices)
selected_indices
```

<br>

- 선택된 10개의 문서를 출력

```python
from langchain_core.documents import Document

selected_docs = [Document(page_content=split_docs[doc]) for doc in selected_indices]
selected_docs
```

<br>

- 이전에 생성한 `map_refine_chain`을 사용하여 요약 생성

```python
refined_summary = map_refine_chain.invoke(selected_docs)
```

<br>

- 최종 결과 출력

```python
print(refined_summary)
```

<br>

<hr>